In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pdfsss3/pdf2.pdf
/kaggle/input/pdfsss3/pdf5.pdf
/kaggle/input/pdfsss3/pdf4.pdf
/kaggle/input/pdfsss3/pdf8.pdf
/kaggle/input/pdfsss3/pdf1.pdf
/kaggle/input/pdfsss3/pdf11.pdf
/kaggle/input/pdfsss3/pdf7.pdf
/kaggle/input/pdfsss3/pdf10.pdf
/kaggle/input/pdfsss3/pdf9.pdf
/kaggle/input/pdfsss3/pdf3.pdf
/kaggle/input/pdfsss3/pdf6.pdf


In [2]:
pip install pdfplumber pandas openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 694.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 59.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pdfplumber
import pandas as pd
import os
from openpyxl import Workbook

# Function to extract text from a PDF, split into words, and store each row as a dictionary
def pdf_to_dicts(pdf_file):
    rows_list = []  # List to hold all rows as dictionaries
    with pdfplumber.open(pdf_file) as pdf:
        # Iterate through pages
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                # Split text by newlines to get individual lines
                lines = text.split('\n')
                for line in lines:
                    # Split each line into words based on spaces
                    words = line.split()
                    # Create a dictionary where keys are word indices and values are the words
                    word_dict = {i: word for i, word in enumerate(words)}
                    # Append the dictionary to the list of rows
                    rows_list.append(word_dict)
    print(f"Extracted data from {pdf_file}:", rows_list)  # DEBUG: Print extracted data
    return rows_list  # Returns a list of dictionaries, each representing a row

# Function to convert the list of dictionaries into a DataFrame
def dicts_to_dataframe(rows_list):
    df = pd.DataFrame(rows_list).fillna('')  # Fill NaNs with empty strings to avoid issues
    print(f"DataFrame before transformation:\n{df}")  # DEBUG: Print the DataFrame before transformation
    return df

# Function to transform the DataFrame based on specific conditions
def transform_dataframe(df):
    pd_rows, pd_cols = df.shape

    def validate_df(row_num, col_num):
        if pd_rows > row_num and pd_cols > col_num:
            return True
        else:
            print(f"Error: DataFrame does not have enough rows ({pd_rows}) or columns ({pd_cols}) for the condition at row {row_num} and column {col_num}.")
            return False

    # CANOPY transformation
    if validate_df(2, 0) and df.iloc[1, 0] == "CANOPY":
        print("Applying CANOPY transformation...")  # DEBUG: Transformation type
        blank_row = pd.Series([''] * pd_cols, index=df.columns)
        df = pd.concat([df, blank_row.to_frame().T], ignore_index=True)

        column_names = [
            'INSURANCE_PROVIDER', 'CHEQUE_#', 'CHEQUE_DATE', 'BATCH_ID', 'DATE_FILLED', 
            'CLAIM_#', 'MEMBER_ID', 'NAME', 'BENEFIT', 'REF._NO', 'STAT', 'CHARGED', 
            'COPAY', 'EXCLUDED', 'DEDUCTIBLE', '%', 'PAYABLE', 'FEE', 'GCT', 'TOTAL', 
            'REJECT_REASON', 'REMARK_CODE'
        ]

        # Adjust the column names to match the number of columns in the DataFrame
        if len(column_names) > pd_cols:
            column_names = column_names[:pd_cols]  # Trim the column names list
        elif len(column_names) < pd_cols:
            column_names.extend([''] * (pd_cols - len(column_names)))  # Pad with empty strings

        column_names_row = pd.Series(column_names, index=df.columns)
        df = pd.concat([df, column_names_row.to_frame().T], ignore_index=True)

        if pd_rows > 10:
            data_to_copy = df.iloc[5:pd_rows-5, 0:13]
            df = pd.concat([df, data_to_copy], ignore_index=True)

        print(f"DataFrame after CANOPY transformation:\n{df}")  # DEBUG: Print DataFrame after transformation
        return df, 'CANOPY'

    # GUARDIAN transformation
    if validate_df(6, 0) and df.iloc[5, 0] == "GUARDIAN":
        print("Applying GUARDIAN transformation...")  # DEBUG: Transformation type
        blank_row = pd.Series([''] * pd_cols, index=df.columns)
        df = pd.concat([df, blank_row.to_frame().T], ignore_index=True)

        column_names = [
            'INSURANCE_PROVIDER', 'CHEQUE_#', 'CHEQUE_DATE', 'BATCH_ID', 'DATE_FILLED', 
            'CLAIM_#', 'MEMBER_ID', 'NAME', 'BENEFIT', 'REF._NO', 'STAT', 'CHARGED', 
            'COPAY', 'EXCLUDED', 'DEDUCTIBLE', '%', 'PAYABLE', 'FEE', 'GCT', 'TOTAL', 
            'REJECT_REASON', 'REMARK_CODE'
        ]

        # Adjust the column names to match the number of columns in the DataFrame
        if len(column_names) > pd_cols:
            column_names = column_names[:pd_cols]  # Trim the column names list
        elif len(column_names) < pd_cols:
            column_names.extend([''] * (pd_cols - len(column_names)))  # Pad with empty strings

        column_names_row = pd.Series(column_names, index=df.columns)
        df = pd.concat([df, column_names_row.to_frame().T], ignore_index=True)

        if pd_rows > 17:
            data_to_copy = df.iloc[10:pd_rows-7, 0:14]
            df = pd.concat([df, data_to_copy], ignore_index=True)

        print(f"DataFrame after GUARDIAN transformation:\n{df}")  # DEBUG: Print DataFrame after transformation
        return df, 'GUARDIAN'

    # SAGICOR transformation
    if validate_df(1, 0) and df.iloc[0, 0] == "Sagicor":
        print("Applying SAGICOR transformation...")  # DEBUG: Transformation type
        blank_row = pd.Series([''] * pd_cols, index=df.columns)
        df = pd.concat([df, blank_row.to_frame().T], ignore_index=True)

        column_names = [
            'INSURANCE_PROVIDER', 'CHEQUE_#', 'CHEQUE_DATE', 'BATCH_ID', 'DATE_FILLED', 
            'CLAIM_#', 'MEMBER_ID', 'NAME', 'BENEFIT', 'REF._NO', 'STAT', 'CHARGED', 
            'COPAY', 'EXCLUDED', 'DEDUCTIBLE', '%', 'PAYABLE', 'FEE', 'GCT', 'TOTAL', 
            'REJECT_REASON', 'REMARK_CODE'
        ]

        # Adjust the column names to match the number of columns in the DataFrame
        if len(column_names) > pd_cols:
            column_names = column_names[:pd_cols]  # Trim the column names list
        elif len(column_names) < pd_cols:
            column_names.extend([''] * (pd_cols - len(column_names)))  # Pad with empty strings

        column_names_row = pd.Series(column_names, index=df.columns)
        df = pd.concat([df, column_names_row.to_frame().T], ignore_index=True)

        rows_to_delete = ["PAGE", "PROPHYSIO", "134CONSTANTSPRINGROAD", "KINGSTON10", 
                          "PROVIDERTAXID:", "28-48BarbadosAvenue.,Kingston5,JamaicaW.I.", 
                          "Telephone:(876)929-8920-9", "YOU", "-"]
        df = df[~df.iloc[:, 0].str.startswith(tuple(rows_to_delete), na=False)]
        df = df.dropna(how='all').reset_index(drop=True)

        print(f"DataFrame after SAGICOR transformation:\n{df}")  # DEBUG: Print DataFrame after transformation
        return df, 'SAGICOR'

    print("No transformation matched.")  # DEBUG: No transformation case
    print(f"DataFrame remains unchanged:\n{df}")  # DEBUG: Print DataFrame if no transformation
    return df, None  # No valid transformation

# Process PDFs to Excel with transformations, saving each transformation to a separate sheet
def process_pdfs_to_excel(pdf_files, output_directory, excel_file):
    wb = Workbook()

    for pdf_file in pdf_files:
        # Convert PDF text into dictionaries
        rows_list = pdf_to_dicts(pdf_file)

        # Check if rows_list is empty
        if not rows_list:
            print(f"No data extracted from {pdf_file}")  # DEBUG: Empty data case
            continue

        # Convert the dictionaries into a DataFrame
        df = dicts_to_dataframe(rows_list)

        # Apply transformations based on conditions
        transformed_df, sheet_name = transform_dataframe(df)

        if sheet_name:
            # Add the transformed DataFrame to a separate sheet in the Excel file
            ws = wb.create_sheet(sheet_name)
            for r in transformed_df.itertuples(index=False):
                ws.append(r)
            print(f"Data from {pdf_file} has been transformed and saved to sheet '{sheet_name}'")
        else:
            # If no transformation, still save the untransformed data in the sheet
            sheet_name = os.path.basename(pdf_file).replace('.pdf', '')  # Use file name as sheet name
            ws = wb.create_sheet(sheet_name)
            for r in df.itertuples(index=False):
                ws.append(r)
            print(f"No transformation applied, saved raw data to sheet '{sheet_name}'")

    # Save the workbook with all the sheets
    wb.save(excel_file)
    print(f"All PDF files have been processed and saved to {excel_file}")

# Paths to the PDF files (replace with your actual file paths)
pdf_files = [
    "/kaggle/input/pdfsss3/pdf1.pdf",
    "/kaggle/input/pdfsss3/pdf2.pdf",
    "/kaggle/input/pdfsss3/pdf3.pdf",
    "/kaggle/input/pdfsss3/pdf4.pdf",
    "/kaggle/input/pdfsss3/pdf5.pdf",
    "/kaggle/input/pdfsss3/pdf6.pdf",
    "/kaggle/input/pdfsss3/pdf7.pdf",
    "/kaggle/input/pdfsss3/pdf8.pdf",
    "/kaggle/input/pdfsss3/pdf9.pdf",
    "/kaggle/input/pdfsss3/pdf10.pdf",
    "/kaggle/input/pdfsss3/pdf11.pdf"
]

# Directory where the converted CSVs will be saved
output_directory = "/kaggle/working/"

# Path to the final Excel file where the data will be stored
excel_file = "/kaggle/working/imported_data.xlsx"

# Run the process to convert PDFs to Excel and apply transformations
process_pdfs_to_excel(pdf_files, output_directory, excel_file)


Extracted data from /kaggle/input/pdfsss3/pdf1.pdf: [{0: 'Provider', 1: 'Cheque', 2: 'Details'}, {0: 'CANOPY', 1: 'INSURANCE', 2: 'LIMITED.'}, {0: 'Provider', 1: ':', 2: 'PRO', 3: 'PHYSIO', 4: '-', 5: 'APH02315', 6: 'Cheque', 7: 'N0./Dt.', 8: ':', 9: '7390401', 10: '/', 11: '20-AUG-2024'}, {0: 'Batch', 1: 'ID', 2: ':', 3: '9088'}, {0: 'Dt.', 1: 'Filled', 2: 'Claim#', 3: 'Member', 4: 'ID', 5: 'Name', 6: 'Benefit', 7: 'Stat', 8: 'Charged', 9: 'Copay', 10: 'Payable', 11: 'Fee', 12: 'GCT', 13: 'Total'}, {0: '08-AUG-2024', 1: '0007618', 2: '7217197599984', 3: 'MELANIE', 4: 'BITTER', 5: 'PHYSIO', 6: 'P', 7: '8,000.00', 8: '6,000.00', 9: '2,000.00', 10: '35.00', 11: '5.25', 12: '1,959.75'}, {0: '08-AUG-2024', 1: '0007620', 2: '7207522899984', 3: 'CAREEN', 4: 'NOLAN', 5: 'PHYSIO', 6: 'P', 7: '8,000.00', 8: '4,800.00', 9: '3,200.00', 10: '56.00', 11: '8.40', 12: '3,135.60'}, {0: '08-AUG-2024', 1: '0007621', 2: '7475204359984', 3: 'CAREEN', 4: 'NOLAN', 5: 'PHYSIO', 6: 'P', 7: '4,800.00', 8: '3,8